# Module 3 Homework: Data Warehousing & BigQuery
In this homework we'll practice working with BigQuery and Google Cloud Storage.
File


## Data
For this homework we will be using the Yellow Taxi Trip Records for January 2024 - June 2024 (not the entire year of data).

Parquet Files are available from the New York City Taxi Data found here:

https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page


## BigQuery Setup
Create an external table using the Yellow Taxi Trip Records.

Create a (regular/materialized) table in BQ using the Yellow Taxi Trip Records (do not partition or cluster this table).



## Question 1. Counting records
What is count of records for the 2024 Yellow Taxi Data?
- 20,332,093

```sql
SELECT COUNT(*) 
FROM `project_id.dataset_id.yellow_taxi_replica`;



## Question 2. Data read estimation
Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.

What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?
```sql
-- Count distinct PULocationIDs from the external table
SELECT COUNT(DISTINCT PULocationID) 
FROM `project_id.dataset_id.yellow_taxi_external`;
```
Duration
0 sec
Bytes processed
0 B (results cached)
Bytes billed
0 B

```sql
-- Count distinct PULocationIDs from the materialized table
SELECT COUNT(DISTINCT PULocationID)
FROM `project_id.dataset_id.yellow_taxi_replica`;

```
Duration
0 sec
Bytes processed
155.12 MB
Bytes billed
156 MB

- 0 MB for the External Table and 155.12 MB for the Materialized Table



## Question 3. Understanding columnar storage
Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. Now write a query to retrieve the PULocationID and DOLocationID on the same table.

Why are the estimated number of Bytes different?

- BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.

Bytes processed
155.12 MB 
for PULocationID
Bytes processed
310.24 MB
PULocationID and DOLocationID
```sql
SELECT PULocationID, DOLocationID
FROM `project_id.dataset_id.yellow_taxi_replica`;
```



## Question 4. Counting zero fare trips
How many records have a fare_amount of 0?
- 8333
```sql
SELECT COUNT(*)
FROM `project_id.dataset_id.yellow_taxi_replica`
WHERE fare_amount = 0;
```

## Question 5. Partitioning and clustering
What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

- Partition by tpep_dropoff_datetime and Cluster on VendorID

- Partition by tpep_dropoff_datetime: This optimizes queries that filter by this date column, as partitioning reduces the data scanned by focusing only on relevant partitions.

- Cluster on VendorID: This optimizes queries that order by VendorID, allowing BigQuery to efficiently sort and read only the necessary data within each partition.

```sql
CREATE OR REPLACE TABLE project_id.dataset_id.yellow_taxi_optimised
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY VendorID AS
SELECT * FROM project_id.dataset_id.yellow_taxi_replica;
```

## Question 6. Partition benefits
Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive)

Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values?

Choose the answer which most closely matches.

- 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

```sql

SELECT DISTINCT VendorID
FROM `project_id.dataset_id.yellow_taxi_replica`
WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15';
```
Bytes processed
310.24 MB

```sql
SELECT DISTINCT VendorID
FROM `project_id.dataset_id.yellow_taxi_optimised`
WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15';
```
Bytes processed
26.84 MB

## Question 7. External table storage
Where is the data stored in the External Table you created?


- GCP Bucket


The data is not physically stored inside BigQuery but instead in an external storage source (usually Google Cloud Storage buckets). This allows to query large datasets directly without needing to load the data into BigQuery storage.


## Question 8. Clustering best practices
It is best practice in Big Query to always cluster your data:


- False

Clustering is not always required in BigQuery, and it should be used only when needed. Clustering helps optimize queries that filter or sort by specific columns, but it can incur additional storage costs and overhead.


## Question 9. Understanding table scans
No Points: Write a SELECT count(*) query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

Bytes processed
0 B (results cached)
"0 bytes (results cached)" because BigQuery didn't re-scan the data due to result caching.
```sql
SELECT COUNT(*) 
FROM `project_id.dataset_id.yellow_taxi_replica`;
```

